### Quickstart: Compare runs, shoose a model, and del=ploy it to a REST API

In this, we will:

- Run a hyperparameter sweep on a training script
- Compare the result of the runs in the MLflow UI
- Choose the best run and register it as a model
- Deploy the model to a REST API
- Build a container image suitable for deployment to a cloud platform


In [9]:
from tensorflow import keras
import numpy as np
import pandas as pd
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import mlflow
from mlflow.models import infer_signature

In [10]:
data = pd.read_csv("https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv", sep=";")
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [11]:
## split data into training and test sets
train, test = train_test_split(data, test_size=0.2, random_state=42)
train_X = train.drop("quality", axis=1)
train_y = train["quality"]

test_X = test.drop("quality", axis=1)
test_y = test["quality"]

## splitting this train into train and validation
train_X, val_X, train_y, val_y = train_test_split(train_X, train_y, test_size=0.2, random_state=42)

signature = infer_signature(train_X, train_y)

/opt/anaconda3/envs/krish/lib/python3.11/site-packages/mlflow/types/utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


In [12]:
## ANN Model
def train_model(params, epochs, train_x, train_y, val_x, val_y, test_x, test_y ):

    ## Define model architecture
    mean = np.mean(train_X, axis=0)
    var = np.var(train_X, axis=0)

    model = keras.Sequential([
        keras.Input([train_X.shape[1]]),
        keras.layers.Normalization(mean=mean, variance=var),
        keras.layers.Dense(64, activation="relu"),
        keras.layers.Dense(1)
    ])

    ## compile the model
    model.compile(loss="mse", 
                  optimizer=keras.optimizers.SGD(learning_rate=params["lr"], 
                                                 momentum=params["momentum"]), 
                                                 metrics=[keras.metrics.RootMeanSquaredError()])  
    
    with mlflow.start_run(nested=True):
        model.fit(train_x, train_y, validation_data=(val_x, val_y),
                  epochs=epochs,
                  batch_size=64)
        
        ## Evaluate the model
        eval_result = model.evaluate(test_x, test_y, batch_size=64)

        eval_rmse = eval_result[1]

        ## Log the parameters and results
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse", eval_rmse)

        ## Log the model
        mlflow.tensorflow.log_model(model=model, artifact_path="model", signature=signature)

        return {'loss': eval_rmse, "status": STATUS_OK, "model": model}
        

In [13]:
def objective(params):
    ## MLFLOW will track the parameters and results for each run
    result = train_model(params,
                         epochs=3,
                         train_x=train_X,
                         train_y=train_y,
                         val_x=val_X,
                         val_y=val_y,
                         test_x=test_X,
                         test_y=test_y)
    return result

In [14]:
space={
    "lr":hp.loguniform("lr", np.log(1e-5), np.log(1e-1)),
    "momentum":hp.uniform("momentum", 0.0, 1.0)
}

In [ ]:
mlflow.set_experiment("/wine-quality")
with mlflow.start_run() as run:
    # conduct the hyperparameter search using hyperopt
    trials=Trials()
    best=fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=4, trials=trials)

    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # Log the best parameters, loss, model
    mlflow.log_params(best)
    mlflow.log_metric("best_loss", best_run["loss"])

    mlflow.tensorflow.log_model(best_run["model"], artifact_path="model", signature=signature)

    # print out the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")

  0%|          | 0/4 [00:00<?, ?trial/s, best loss=?]

Epoch 1/3                                            

49/49 [==============================] - 0s 2ms/step - loss: 29.3662 - root_mean_squared_error: 5.4191 - val_loss: 25.1852 - val_root_mean_squared_error: 5.0185

Epoch 2/3                                            

49/49 [==============================] - 0s 751us/step - loss: 21.8217 - root_mean_squared_error: 4.6714 - val_loss: 18.7577 - val_root_mean_squared_error: 4.3310

Epoch 3/3                                            

49/49 [==============================] - 0s 733us/step - loss: 16.1629 - root_mean_squared_error: 4.0203 - val_loss: 13.9559 - val_root_mean_squared_error: 3.7358

16/16 [==============================] - 0s 479us/step - loss: 13.9284 - root_mean_squared_error: 3.7321

  0%|          | 0/4 [00:00<?, ?trial/s, best loss=?]

ERROR:hyperopt.fmin:job exception: All fields in the model signature's input schema must be of type TensorSpec.



  0%|          | 0/4 [00:00<?, ?trial/s, best loss=?]


MlflowException: All fields in the model signature's input schema must be of type TensorSpec.